<img src="https://industrial.uniandes.edu.co/sites/default/files/imagenes/uniandeslogo.png" alt="Universidad de los Andes" style="float: right; width: 300px; height: auto;">

# Taller 1 - Ejercicio 2

Econometría Avanzada, 2026-1  

Febrero de 2026

````
Aclaración: En este cuaderno se utilizo IA generativa, Claude, para ordenar, unificar y comentar el código y crear las tablas que se exportan a .tex
````

### Importar librerías

Carga las librerías necesarias para el análisis.

**Librerías utilizadas:**
- `pandas` y `numpy`: manipulación y análisis de datos
- `statsmodels`: estimación OLS con errores estándar robustos y clusterizados
- `yaml` y `pathlib`: gestión de rutas de archivos
- `warnings`: control de advertencias

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
import yaml
from pathlib import Path

### Configuración de rutas

Carga los directorios principales desde el archivo de configuración `paths.yml`.

In [2]:
with open('paths.yml', 'r') as f:
    paths = yaml.safe_load(f)

raw    = Path(paths['data']['raw'])
tables = Path(paths['outputs']['tables'])
tables.mkdir(parents=True, exist_ok=True)

docs_tables = Path(paths['project']['root']) / 'docs' / 'tables'

### Configuración de opciones

Suprime advertencias durante la ejecución.

In [3]:
warnings.filterwarnings('ignore')
np.random.seed(31416)

### Lectura de datos

Lee la base de datos `race_teaching.dta`, que corresponde al experimento de Gershenson et al. (2022). Cada observación es un estudiante de raza negra que participó en el experimento de asignación aleatoria de profesores en kínder.

In [4]:
df = pd.read_stata(raw / 'race_teaching.dta')


## Punto 5. Regresiones con errores robustos y clusterizados

Estima el modelo:

$$Y_i = \beta_0 + \beta_1 D_i + \varepsilon_i$$

para dos variables de resultado: el puntaje en matemáticas del SAT y la probabilidad de ingresar a la universidad. Se reportan errores estándar robustos a heterocedasticidad (HC1) y clusterizados a nivel de colegio.

### Función de estimación OLS

Función auxiliar que estima OLS y devuelve el modelo con los errores estándar especificados.

In [5]:
def run_ols(y_col, df, cov_type, cluster_col=None):
    y    = df[y_col].astype(float)
    X    = sm.add_constant(df[['D']].astype(float))
    mask = y.notna() & X.notna().all(axis=1)
    if cluster_col is not None:
        mask = mask & df[cluster_col].notna()
    y_, X_ = y[mask], X[mask]
    if cov_type == 'cluster':
        model = sm.OLS(y_, X_).fit(
            cov_type='cluster',
            cov_kwds={'groups': df.loc[mask, cluster_col].values}
        )
    else:
        model = sm.OLS(y_, X_).fit(cov_type=cov_type)
    return model, int(mask.sum())


def add_stars(pval):
    if pval < 0.01:   return '***'
    elif pval < 0.05: return '**'
    elif pval < 0.1:  return '*'
    return ''

### Estimación del modelo

Estima el modelo para las dos variables de resultado con errores estándar HC1 y clusterizados a nivel de colegio.

In [6]:
outcomes = {
    'sat_math': 'SAT Matemáticas',
    'college':  'Asistencia universitaria'
}

results_p5 = {}
for var, label in outcomes.items():
    m_hc, n_hc  = run_ols(var, df, cov_type='HC1')
    m_cl, n_cl  = run_ols(var, df, cov_type='cluster', cluster_col='school')
    results_p5[label] = {
        'coef':    m_hc.params['D'],
        'pval_hc': m_hc.pvalues['D'],
        'se_hc':   m_hc.bse['D'],
        'se_cl':   m_cl.bse['D'],
        'pval_cl': m_cl.pvalues['D'],
        'N':       n_hc
    }

### Tabla: Efectos del profesor negro sobre resultados educativos

Construye y exporta la tabla de resultados para el punto 5. El coeficiente reportado corresponde a $\beta_1$ (efecto de tener un profesor negro en kínder). Los errores estándar se reportan en paréntesis bajo el coeficiente.

In [7]:
rows_p5 = []
for label, r in results_p5.items():
    stars = add_stars(r['pval_hc'])
    rows_p5.append({'Fila': 'Tiene prof. negro', label: f"{r['coef']:.4f}{stars}"})
    rows_p5.append({'Fila': '(SE Robusto HC1)', label: f"({r['se_hc']:.4f})"})
    rows_p5.append({'Fila': '[SE Cluster colegio]', label: f"[{r['se_cl']:.4f}]"})
    rows_p5.append({'Fila': 'N', label: str(r['N'])})

df_p5 = (
    pd.DataFrame(rows_p5)
    .groupby('Fila', sort=False)
    .first()
    .reset_index()
    .set_index('Fila')
)

idx_order = ['Tiene prof. negro', '(SE Robusto HC1)', '[SE Cluster colegio]', 'N']
df_p5 = df_p5.loc[idx_order]

latex_p5 = df_p5.to_latex(
    caption='Efectos de tener un profesor negro en kínder sobre resultados educativos',
    label='tab:p5_efectos',
    escape=False,
    column_format='lrr',
    position='htbp'
)

nota_p5 = (
    '\\begin{tablenotes}\\small\n'
    '  \\item *** p<0.01, ** p<0.05, * p<0.1. '
    'Errores estándar robustos a heterocedasticidad (HC1) entre paréntesis. '
    'Errores estándar clusterizados a nivel de colegio entre corchetes. '
    'Significancia basada en errores estándar robustos.\n'
    '\\end{tablenotes}\n'
)
latex_p5 = latex_p5.replace('\\end{table}', nota_p5 + '\\end{table}')

with open(tables / 'tabla_p5_efectos.tex', 'w', encoding='utf-8') as f:
    f.write(latex_p5)


## Punto 6. Sensibilidad de errores estándar al nivel de clusterización

Repite la estimación del punto 5 para la variable SAT Matemáticas, reportando errores estándar robustos y clusterizados a diferentes niveles de agregación: colegio, condado y estado.

### Estimación con múltiples niveles de cluster

Estima el modelo con la variable de puntaje SAT Matemáticas y diferentes niveles de clusterización.

In [8]:
m_hc_6, n_6  = run_ols('sat_math', df, cov_type='HC1')
m_sc_6, _    = run_ols('sat_math', df, cov_type='cluster', cluster_col='school')
m_co_6, _    = run_ols('sat_math', df, cov_type='cluster', cluster_col='county')
m_st_6, _    = run_ols('sat_math', df, cov_type='cluster', cluster_col='state')

coef_6 = m_hc_6.params['D']
stars_6 = add_stars(m_hc_6.pvalues['D'])

### Tabla: Sensibilidad de errores estándar al nivel de clusterización

Construye y exporta la tabla con los resultados de la estimación a distintos niveles de agregación.

In [ ]:
rows_p6 = [
    {'Especificación': 'Tiene prof. negro',       'SAT Matemáticas': f"{coef_6:.4f}{stars_6}"},
    {'Especificación': '(SE Robusto HC1)',          'SAT Matemáticas': f"({m_hc_6.bse['D']:.4f})"},
    {'Especificación': '[SE Cluster colegio]',       'SAT Matemáticas': f"[{m_sc_6.bse['D']:.4f}]"},
    {'Especificación': '[SE Cluster condado]',       'SAT Matemáticas': f"[{m_co_6.bse['D']:.4f}]"},
    {'Especificación': '[SE Cluster estado]',        'SAT Matemáticas': f"[{m_st_6.bse['D']:.4f}]"},
    {'Especificación': 'N',                          'SAT Matemáticas': str(n_6)}
]

df_p6 = pd.DataFrame(rows_p6).set_index('Especificación')

latex_p6 = df_p6.to_latex(
    caption='Sensibilidad de errores estándar al nivel de clusterización  SAT Matemáticas',
    label='tab:p6_clusters',
    escape=False,
    column_format='lr',
    position='htbp'
)

nota_p6 = (
    '\\begin{tablenotes}\\small\n'
    '  \\item *** p<0.01, ** p<0.05, * p<0.1. '
    'Errores estándar robustos a heterocedasticidad (HC1) entre paréntesis. '
    'Errores estándar clusterizados entre corchetes. '
    'Significancia basada en errores estándar robustos. '
    'La variable id\_salon no fue incluida en la base de datos.\n'
    '\\end{tablenotes}\n'
)
latex_p6 = latex_p6.replace('\\end{table}', nota_p6 + '\\end{table}')

with open(tables / 'tabla_p6_clusters.tex', 'w', encoding='utf-8') as f:
    f.write(latex_p6)


## Punto 7. Simulación de Susan Athey

Replica la simulación de Monte Carlo del código de Susan Athey. El objetivo es comparar la cobertura de los intervalos de confianza al 95% construidos con errores estándar robustos a heterocedasticidad versus clusterizados a nivel de colegio, en un contexto donde el efecto verdadero del tratamiento es cero.

**Diseño de la simulación:**
- Población de 10 millones de individuos distribuidos en 100 colegios (100,000 por colegio)
- Los primeros 50 colegios tienen un efecto por colegio de +1; los últimos 50, de -1
- El efecto promedio del tratamiento es cero por construcción
- La correlación intra-colegio surge porque `school_effect` es compartido por todos los estudiantes del mismo colegio
- Se toman 1,000 muestras del 1% de la población (~100,000 observaciones por muestra)

### Parámetros de la simulación

Define los parámetros siguiendo el código R de Susan Athey.

In [10]:
rng = np.random.default_rng(31416)

n_schools       = 100
stud_per_school = 100_000
p_sample        = 0.01
n_sim           = 1_000

school_effects = np.where(np.arange(n_schools) < 50, 1.0, -1.0)

### Simulación de Monte Carlo

Para cada iteración, genera una muestra de la población, estima el modelo por MCO y calcula errores estándar robustos (HC1) y clusterizados. Determina si el intervalo de confianza al 95% incluye el cero (efecto verdadero).

La muestra se construye directamente por colegio, sin necesidad de almacenar la población completa en memoria.

In [11]:
se_hc_list  = []
se_cl_list  = []
cover_hc    = 0
cover_cl    = 0

for _ in range(n_sim):
    D_parts      = []
    Y_parts      = []
    school_parts = []

    for s in range(n_schools):
        eff = school_effects[s]
        n_s = rng.binomial(stud_per_school, p_sample)
        if n_s == 0:
            continue
        D_s = rng.integers(0, 2, size=n_s).astype(float)
        U_s = rng.random(size=n_s)
        Y_s = eff * D_s + U_s
        D_parts.append(D_s)
        Y_parts.append(Y_s)
        school_parts.append(np.full(n_s, s, dtype=np.int32))

    D      = np.concatenate(D_parts)
    Y      = np.concatenate(Y_parts)
    school = np.concatenate(school_parts)
    n      = len(D)

    # OLS con constante: Y = b0 + b1*D + eps
    D_mean = D.mean()
    Y_mean = Y.mean()
    D_dem  = D - D_mean
    SXX    = (D_dem ** 2).sum()

    beta1 = (D_dem * Y).sum() / SXX
    beta0 = Y_mean - beta1 * D_mean
    eps   = Y - beta0 - beta1 * D

    # Varianza HC1
    k        = 2
    hc1_var  = (n / (n - k)) * (D_dem ** 2 * eps ** 2).sum() / SXX ** 2
    se_hc    = np.sqrt(hc1_var)

    # Varianza cluster (colegio), con corrección de muestra finita
    scores         = D_dem * eps
    sort_idx       = np.argsort(school, kind='stable')
    school_sorted  = school[sort_idx]
    scores_sorted  = scores[sort_idx]
    unique_s, cts  = np.unique(school_sorted, return_counts=True)
    G              = len(unique_s)
    starts         = np.concatenate([[0], cts.cumsum()[:-1]])
    score_by_s     = np.add.reduceat(scores_sorted, starts)
    raw_cl_var     = (score_by_s ** 2).sum() / SXX ** 2
    cl_var         = (G / (G - 1)) * ((n - 1) / (n - k)) * raw_cl_var
    se_cl          = np.sqrt(cl_var)

    # Intervalos de confianza al 95%
    z = 1.96
    ci_hc = (beta1 - z * se_hc, beta1 + z * se_hc)
    ci_cl = (beta1 - z * se_cl, beta1 + z * se_cl)

    cover_hc += int(ci_hc[0] <= 0.0 <= ci_hc[1])
    cover_cl += int(ci_cl[0] <= 0.0 <= ci_cl[1])

    se_hc_list.append(se_hc)
    se_cl_list.append(se_cl)

### Resultados y tabla de cobertura

Calcula el error estándar promedio y el porcentaje de cobertura para cada tipo de error estándar. Actualiza la tabla de cobertura en el documento LaTeX.

In [12]:
se_hc_mean   = np.mean(se_hc_list)
se_cl_mean   = np.mean(se_cl_list)
var_hc_mean  = np.mean(np.array(se_hc_list) ** 2)
var_cl_mean  = np.mean(np.array(se_cl_list) ** 2)
pct_hc       = cover_hc / n_sim * 100
pct_cl       = cover_cl / n_sim * 100

In [13]:
cover_tex = f"""\\begin{{table}}[htbp]
	\\centering
	\\begin{{tabular}}{{cccc}}
		\\hline
		\\multicolumn{{2}}{{c}}{{Heterocedasticidad}} & \\multicolumn{{2}}{{c}}{{Cluster}} \\\\
		\\midrule
		$\\hat{{\\mathbb{{V}}}}$ Promedio & $\\%$ Cobertura & $\\hat{{\\mathbb{{V}}}}$ Promedio & $\\%$ Cobertura \\\\
		\\midrule
		{var_hc_mean:.6f} & {pct_hc:.1f}\\% & {var_cl_mean:.6f} & {pct_cl:.1f}\\% \\\\
		\\hline
	\\end{{tabular}}
	\\caption{{Cobertura teórica de heterocedasticidad vs clusters}}
\\end{{table}}
"""

with open(docs_tables / 'cover_rate.tex', 'w', encoding='utf-8') as f:
    f.write(cover_tex)